In [19]:
#loading the data
import pandas as pd
from surprise import Dataset
from surprise import Reader
from sklearn.utils import shuffle

train_ratings = pd.read_csv("train_ratings.csv")
val_ratings = pd.read_csv("val_ratings.csv")
test_ratings = pd.read_csv("test_ratings.csv")

shuffled_ratings = shuffle(train_ratings)

reader = Reader(rating_scale=(0.5, 5))

train_data = Dataset.load_from_df(shuffled_ratings[['userId', 'movieId', 'rating']], reader=reader)
trainset = train_data.build_full_trainset()

val_data = Dataset.load_from_df(val_ratings[['userId', 'movieId', 'rating']], reader=reader)
#valset = val_data.build_full_trainset()

In [ ]:
# #hyperparameter tuning
# from surprise.model_selection import GridSearchCV
# import time
# start = time.time()

# param_grid = {'n_epochs': [20, 30], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.0, 0.2, 0.4]}
# gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

# gs.fit(train_data)

# # best RMSE score
# print(gs.best_score['rmse'])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params['rmse'])

# print(time.time() - start)

In [ ]:
from surprise import SVD
from surprise import SVDpp

#fitting the training set
algo = SVD()
#algo = SVDpp() #uncomment and comment above line for SVDpp
algo.fit(trainset)

In [ ]:
#validating
from surprise.model_selection import train_test_split
from surprise import accuracy

#uses the validation set for validation
junk, valset = train_test_split(val_data, train_size=1)
evaluations = algo.test(valset)
accuracy.rmse(evaluations, verbose=True)

In [ ]:
#predicting
import csv
csvfile = open('submission.csv', 'wb')
csvwriter = csv.writer(csvfile)
csvwriter.writerow(["Id", "rating"])

for row in test_ratings.itertuples():
    predictions = algo.predict(row.userId, row.movieId)
    #round to nearest 0.5 (performed worse)
    #rating = round(predictions[3] * 2) / 2
    #csvwriter.writerow([row.Id, rating])
    csvwriter.writerow([row.Id, predictions[3]])
csvfile.close()